# ERCPy: 2D Gaussian fit demonstration notebook

#### by Martial Duchamp
Laboratory for in situ and operando nanoscopy
Nanyang Technological University
mduchamp@ntu.edu.sg

### Absract:
The notebook demonstrates usage of the bulk plasmon function to fit EELS spectra.

## Preamble:

#### Plotting outside the notebook to enable interactions with user:

In [1]:
%matplotlib qt5

In [2]:
import hyperspy.api as hs

/root/anaconda3/lib/python3.8/site-packages/pyUSID/viz/__init__.py:16: FutureWarning: Please use sidpy.viz.plot_utils instead of pyUSID.viz.plot_utils. pyUSID.plot_utils will be removed in a future release of pyUSID
  warn('Please use sidpy.viz.plot_utils instead of pyUSID.viz.plot_utils. '


In [3]:
import os
import scipy as sc
import numpy as np

In [4]:
import matplotlib.pyplot as plt

## Loading data:

In [7]:
path="/home/martial/OneDrive/Data_NTU/Lan/Perovskite_transistor_Nripan/SI10/"

In [17]:
file_name="EELS_Spectrum_Image.dm4"
specImg = hs.load(path+file_name)

#### Plotting:

#Crop spatially, if needed
specImg.crop(0, 1.,9.)
specImg.crop(1, 4.,12.)

In [21]:
specImg.plot()

In [22]:
specImg.spikes_removal_tool()

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

In [20]:
specImg.plot()

In [23]:
specImg.align_zero_loss_peak(crop=True)


Initial ZLP position statistics
-------------------------------
Summary statistics
------------------
mean:	-4.38
std:	0.944

min:	-6.1
Q1:	-5.25
median:	-4.35
Q3:	-3.45
max:	-2.4


In [104]:
specImg.plot()

In [103]:
specImg.crop_signal1D(10.,35.)

In [191]:
specImg.axes_manager[2].scale

0.05000000074505806

#remove the calibration, then the coordinates are given in pixel in the plot
min_specImg.axes_manager.show();
#Use the GUI to set scale=1 and origin=0 for both x and y

#remove the calibration, then the coordinates are given in pixel in the plot
specImg.axes_manager.show();
#Use the GUI to set scale=1 and origin=0 for both x and y

#### Curve fitting

In [26]:
from scipy.optimize import curve_fit
import scipy.optimize as opt

In [385]:
def V_bulk(xo, para, scale_ev, offset):
    a = para[0]
    b = para[1]
    c = para[2]
    E_p = para[3]
    Delta_E_p = para[4]
    x_cal = xo
   
    I = a*x_cal/((x_cal**2-E_p**2)**2+(Delta_E_p*x_cal)**2) + b*x_cal+c    
    return I

In [403]:
#Define the twoD_Gaussian for fitting ONLY
def V_bulk_f_fit(x, a, b, c, E_p, Delta_E_p):
    I = a*x/((x**2-E_p**2)**2+(Delta_E_p*x)**2) + b*x+c    
    return I.ravel()

def V_bulk_fit(exp_data, scale_ev, offset):
    b = 0
    c = np.min(exp_data)
    E_p = np.argmax(exp_data)*scale_ev+offset+10
    Delta_E_p =  exp_data.size*scale_ev/3
    a = np.max(exp_data)*1000
    initial_guess = (a, b, c, E_p, Delta_E_p)
    
    # Create x and y indices
    x = np.linspace(0, np.shape(exp_data)[0], np.shape(exp_data)[0])
    u = np.meshgrid(x)
    u = u[0][:]*scale_ev+offset
    popt, pcov = opt.curve_fit(V_bulk_f_fit, u, exp_data.ravel(), p0=initial_guess, maxfev=5000)
    
    return popt

In [413]:
# The fit_around_px value has to be adjusted to get proper fitting
nbr_para_fit = 5 #Number of parameters to be fitted

#Np.array with the results from the 2D Gaussian fit
fit_values = hs.signals.Signal2D(np.zeros((nbr_para_fit,specImg.data.shape[0],specImg.data.shape[1])), \
                                 dtype=np.float32)

scale_ev = specImg.axes_manager[2].scale
offset_ev = specImg.axes_manager[2].offset

for i in range(0,specImg.data.shape[0]): 
    for j in range(0,specImg.data.shape[1]): 
        fit_values.data[:,i,j]=V_bulk_fit(specImg.data[i,0,:], scale_ev, offset_ev)

In [414]:
x_value = np.zeros_like(specImg.data[0,0,:])
for i in range(0,specImg.data[0,0,:].shape[0]):
    x_value[i]=i*scale_ev+offset_ev

In [417]:
fig, ax1 = plt.subplots()
ax1.plot(x_value,V_bulk(x_value[:],fit_values.data[:,20,20], scale_ev, offset_ev), 'r+', label ='Fit', markersize=7, markevery=12)
ax1.plot(x_value,specImg.data[20,20,:], 'b-', label ='Input', markersize=7, markevery=12)

ax1.set_ylabel('Voltage [V]',fontsize=20)
ax1.set_xlabel('Time ' +r'$[\mu s]$',fontsize=20)

#ax2.plot(RGA_data[:,0]-Time_voltage[0,0],RGA_data[:,1], 'o', label='O2')
#ax2.plot(RGA_data[:,0]-Time_voltage[0,0],RGA_data[:,2], 'v', label='H2')
plt.setp(ax1.get_xticklabels(), fontsize=17)
plt.setp(ax1.get_yticklabels(), fontsize=17)
plt.legend(fontsize=17)
fig.tight_layout()
#plt.xlim([-0.9, 5])
#plt.ylim([0, 1.2])
plt.show()
plt.savefig('_2us_pulse.png', dpi=300)

In [412]:
#Energy of the plamson peak of the 1st spectra
fit_values.data[3,0,0]

23.6706255632287